In [15]:
import pandas as pd
import numpy as np
import gym
from gym import spaces
from stable_baselines3 import PPO
from classes import mealplanner

In [16]:
df = pd.read_csv(R'D:\calorie_RL\meal2.csv')
df

,meal_name,ingredients,calories_per_serving,carbs,proteins,fats,diet_type,meal_time,serving_size,region,cost_per_serving_in_inr
0,Rajma Chawal,"Kidney Beans, Rice, Tomato, Onion, Garlic, Spices",385,60.0,12.0,8.0,Vegetarian,NaN,1 medium plate,Punjabi,40.0
1,Chole Bhature,"Chickpeas, Refined Flour, Spices, Onion",450,55.0,15.0,18.0,Vegetarian,NaN,1 plate,Punjabi,50.0
2,Aloo Paratha,"Wheat Flour, Potato, Spices, Butter",290,35.0,6.0,12.0,Vegetarian,NaN,1 paratha,Punjabi,25.0
3,Palak Paneer,"Spinach, Paneer, Onion, Tomato, Spices",300,14.0,16.0,18.0,Vegetarian,NaN,1 bowl,Punjabi,45.0
4,Butter Chicken,"Chicken, Butter, Cream, Tomato, Spices",490,18.0,28.0,32.0,Non-Vegetarian,dinner,1 bowl,Punjabi,80.0
...,...,...,...,...,...,...,...,...,...,...,...
670,Sweet Potato Chaat,"Sweet Potato, Spices, Lemon Juice",180,30.0,3.0,5.0,Vegetarian,Snack,1 bowl,Pan India,30.0
671,"Fruit Salad (Banana, apple, mango,blueberry)","Banana, apple, mango,blueberry",120,30.0,2.0,1.0,vegetarian,Snack,1 bowl,Pan India,60.0
672,Basmati Rice,"Basmati Rice, Water",220,45.0,5.0,2.0,Vegetarian,Lunch,1 cup cooked,Pan India,30.0
673,Quinoa Pulao,"Quinoa, Vegetables, Spices",250,35.0,10.0,8.0,Vegetarian,Lunch,1 bowl,Pan India,45.0


In [19]:
region_map = {'Indian': 0, 'Western': 1, 'Chinese': 2}  # Add all possible regions
diet_map = {'Vegetarian': 0, 'Non-Vegetarian': 1, 'Vegan': 2}  # Add all possible diet types

user_prefs = {
    'calories_required': 2000,
    'region': region_map['Indian'],
    'diet_type': diet_map['Vegetarian'],
    'budget': 3000
}
# Create environment
env = mealplanner.MealPlanEnv(user_prefs, df)

In [20]:
# Train RL agent (PPO)
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

# Generate a meal plan
obs = env.reset()
done = False
meal_plan = []
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    meal_plan.append(env.meal_df.iloc[action])


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


d:\calorie_RL\.venv\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


----------------------------------
| rollout/           |           |
|    ep_len_mean     | 7         |
|    ep_rew_mean     | -1.24e+04 |
| time/              |           |
|    fps             | 1253      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
----------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 7             |
|    ep_rew_mean          | -1.24e+04     |
| time/                   |               |
|    fps                  | 828           |
|    iterations           | 2             |
|    time_elapsed         | 4             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 5.0795497e-06 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -6.51         |
|    explained_variance   | 1.79e-

In [21]:
# Show the personalized meal plan
pd.DataFrame(meal_plan)

,meal_name,ingredients,calories_per_serving,carbs,proteins,fats,diet_type,meal_time,serving_size,region,cost_per_serving_in_inr
277,Mysore Bonda,"Urad Dal, Rice Flour, Spices",120,15.0,3.0,3.0,Vegetarian,snacks,2 bondas,Karnataka,20.0
356,Mahim Halwa,"Semolina, sugar, ghee",300,40.0,4.0,14.0,Vegetarian,Dessert,1 piece,Maharashtra,40.0
264,Vangi Bath,"Rice, Eggplant, Spices",300,35.0,8.0,12.0,Vegetarian,NaN,1 plate,Karnataka,45.0
144,Sooji halwa (Suji Lapsi),"Semolina, Sugar, Ghee, Nuts",250,30.0,5.0,10.0,Vegetarian,dessert,1 bowl,North Indian,40.0
371,Bhindi Masala,"Okra, onion, tomato, spices",140,15.0,4.0,7.0,Vegetarian,Lunch,1 bowl,North India,30.0
389,Kadhi Pakoda,"Yogurt, gram flour, spices, fritters",280,25.0,8.0,15.0,Vegetarian,Lunch,1 bowl,North India,40.0
41,Bhindi Masala,"Okra, Onion, Tomato, Spices, Amchur Powder",140,3.0,10.0,10.0,Vegetarian,NaN,1 bowl,North Indian,22.0
